In [17]:
from pyspark import SparkFiles
import numpy as np
import cv2
import pickle
from subprocess import call

def face_counter(an_img_rdd_element):
    x = an_img_rdd_element[0]
    img = an_img_rdd_element[1]
    
    call('my_c++_exe)')
    img_fname = x.split("/")[-1]
    file_bytes = np.asarray(bytearray(img), dtype=np.uint8)
    im = cv2.imdecode(file_bytes,1)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(SparkFiles.get("haarcascade_frontalface_default.xml"))
    faces = faceCascade.detectMultiScale(im)
    return Row(face_count=len(faces),file_name=img_fname)


In [18]:
!wget https://raw.githubusercontent.com/shantnu/Webcam-Face-Detect/master/haarcascade_frontalface_default.xml > /dev/null 2>&1
sc.addFile("./haarcascade_frontalface_default.xml")

In [13]:
result = sc.binaryFiles("s3a://ps-samples/imdb/images/2003/01/01/*/").map(face_counter).toDF()

In [14]:
result.plot(limit=10)

,face_count,file_name
0,3,img_226.jpg
1,3,img_230.jpg
2,1,img_176.jpg
3,1,img_588.jpg
4,1,img_698.jpg
5,2,img_558.jpg
6,4,img_534.jpg
7,1,img_547.jpg
8,1,img_346.jpg
9,1,img_136.jpg


In [15]:
result.groupBy("face_count").agg(count("face_count").alias("total_files")).sort(desc("total_files")).plot()

,face_count,total_files
0,1,15
1,2,4
2,4,2
3,3,2
4,0,1


In [16]:
result.groupBy("face_count").agg(count("face_count").alias("total_files"))\
.sort(desc("face_count"))\
.plot(kind="bar",x="face_count", y=["total_files"],xTitle="Face Count",yTitle="Number of Files")

In [ ]:
def images(path):
  return sc.binaryFiles(path)